In [26]:
import os.path 

from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

import pickle 

import base64 
import email 
from bs4 import BeautifulSoup
import csv

import gspread
from google.oauth2.service_account import Credentials

In [2]:
SCOPES = ["https://www.googleapis.com/auth/gmail.send",'https://www.googleapis.com/auth/gmail.readonly','https://mail.google.com/', "https://www.googleapis.com/auth/gmail.metadata", "https://www.googleapis.com/auth/gmail.modify"]
client_secret_file = './credenciales/client_secret_python_gmail.json'

## conexion

In [3]:
creds = None
  
# The file token.pickle contains the user access token. 
# Check if it exists 
if os.path.exists('token.pickle'): 
  
    # Read the token from the file and store it in the variable creds 
    with open('token.pickle', 'rb') as token: 
        creds = pickle.load(token) 
  
# If credentials are not available or are invalid, ask the user to log in. 
if not creds or not creds.valid: 
    if creds and creds.expired and creds.refresh_token: 
        creds.refresh(Request()) 
    else: 
        flow = InstalledAppFlow.from_client_secrets_file(client_secret_file, SCOPES) 
        creds = flow.run_local_server(port=0) 
  
    # Save the access token in token.pickle file for the next run 
    with open('token.pickle', 'wb') as token: 
        pickle.dump(creds, token) 
  
# Connect to the Gmail API 
service = build('gmail', 'v1', credentials=creds) 

## requests

#### Ver las etiquetas (LABELS)

In [140]:
results_labels = service.users().labels().list(userId="me").execute().get("labels")
results_labels[0]

{'id': 'CHAT',
 'name': 'CHAT',
 'messageListVisibility': 'hide',
 'labelListVisibility': 'labelHide',
 'type': 'system'}

#### TODO EL LISTADO

In [144]:
# request a list of all the messages 
emailIdList = service.users().messages().list(userId='me').execute().get('messages')
emailIdList[0]

{'id': '18c1bdc54cf53ce2', 'threadId': '18c1bdc54cf53ce2'}

In [147]:
for email_ids in emailIdList: 
    # Get the message from its id 
    email_parts = service.users().messages().get(userId='me', id=email_ids['id']).execute()

{'id': '18ba50e6e973b662',
 'threadId': '18ba50e6e973b662',
 'labelIds': ['IMPORTANT', 'CATEGORY_UPDATES', 'INBOX'],
 'snippet': 'Hi Álvaro, I&#39;m reaching out with an exciting opportunity to join an AI start-up who are building real-time intelligence so businesses can make informed decisions. They have recently gained VC',
 'payload': {'partId': '',
  'mimeType': 'multipart/alternative',
  'filename': '',
  'headers': [{'name': 'Delivered-To', 'value': 'alvarosaezsanchez@gmail.com'},
   {'name': 'Received',
    'value': 'by 2002:a05:7010:934b:b0:393:73cd:86f8 with SMTP id s11csp2649792mdk;        Mon, 6 Nov 2023 06:34:28 -0800 (PST)'},
   {'name': 'X-Google-Smtp-Source',
    'value': 'AGHT+IH9VreP1zpuaubafB7jrxQ8Ii+NqnfCIvjvq1F1QL6VSbp0kpQGr770v9DsP31esPUNYqXf'},
   {'name': 'X-Received',
    'value': 'by 2002:a92:cd88:0:b0:350:f2a0:5c44 with SMTP id r8-20020a92cd88000000b00350f2a05c44mr38554103ilb.2.1699281268597;        Mon, 06 Nov 2023 06:34:28 -0800 (PST)'},
   {'name': 'ARC-Sea

In [154]:
email_parts["labelIds"]

['IMPORTANT', 'CATEGORY_UPDATES', 'INBOX']

#### POR TITULO

In [135]:
emailIdList_filter_by_title = service.users().messages().list(userId='me', q=f'subject:test gmail csv').execute().get('messages')

In [136]:
for email_ids in emailIdList_filter_by_title: 
    # Get the message from its id 
    email_parts = service.users().messages().get(userId='me', id=email_ids['id']).execute()

### obtener campos del email

##### Queries para poner dentro del method list (como en el siguiente ejemplo de subject):
  - subject:
  - from:
  - to:
  - cc:
  - bcc:
  - label:
  - has:attachment
  - has:drive
  - has:document
  - has:spreadsheet
  - has:presentation
  - filename:
  - newer_than:2d
  - deliveredto:
  - OR or { }: --> from:amy OR from:david

https://support.google.com/mail/answer/7190?hl=en    --> All the terms and REGEXtation

In [128]:
emailIdList_filter_by_title = service.users().messages().list(userId='me', q=f'subject:test gmail csv').execute().get('messages')

In [129]:
for email_ids in emailIdList_filter_by_title: 
    # Get the message from its id 
    email_parts = service.users().messages().get(userId='me', id=email_ids['id']).execute()
    email_field_from = [name['value'] for name in email_parts['payload']['headers'] if name['name'] == 'From']
    email_field_to = [name['value'] for name in email_parts['payload']['headers'] if name['name'] == 'To']
    email_field_subject = [name['value'] for name in email_parts['payload']['headers'] if name['name'] == 'Subject']
    email_field_date = [name['value'] for name in email_parts['payload']['headers'] if name['name'] == 'Date'] 

In [130]:
email_field_date

['Wed, 29 Nov 2023 12:47:52 +0100']

In [131]:
email_field_from

['Alv Sz <alvarosaezsanchez@gmail.com>']

### adjuntos

In [173]:
emailIdList_filter_by_title = service.users().messages().list(userId='me', q=f'subject:test gmail excel').execute().get('messages')

In [174]:
for email_ids in emailIdList_filter_by_title: 
    email_parts = service.users().messages().get(userId='me', id=email_ids['id']).execute()

In [186]:
for part in email_parts['payload']['parts']:
    if part['filename'] == "GEO_IP_1_comb (1).xlsx":
        print(part)
        # Decodificar el contenido del archivo adjunto
        file_data = base64.urlsafe_b64decode(part['body']['data'])

        # Guardar el archivo en el sistema local con la extensión correcta (por ejemplo, '.xlsx' para Excel)
        file_name = part['filename']
        if not file_name.endswith('.xlsx'):
            file_name += '.xlsx'

        with open(file_name, 'wb') as f:
            f.write(file_data)

print(f"El archivo {file_name} se ha descargado exitosamente.")

{'partId': '1', 'mimeType': 'application/vnd.openxmlformats-officedocument.spreadsheetml.sheet', 'filename': 'GEO_IP_1_comb (1).xlsx', 'headers': [{'name': 'Content-Type', 'value': 'application/vnd.openxmlformats-officedocument.spreadsheetml.sheet; name="GEO_IP_1_comb (1).xlsx"'}, {'name': 'Content-Disposition', 'value': 'attachment; filename="GEO_IP_1_comb (1).xlsx"'}, {'name': 'Content-Transfer-Encoding', 'value': 'base64'}, {'name': 'X-Attachment-Id', 'value': 'f_lpk0230b0'}, {'name': 'Content-ID', 'value': '<f_lpk0230b0>'}], 'body': {'attachmentId': 'ANGjdJ893tXsXNHHfcbipbGkW-YV5BjCWd5tLB6ZDxivHqzGNF-aQ_XdUhybfdZDNQMIlLGK6hqOaF3Rbl3eZ8wxp8I9a2bQU1-uysG-fsZ9Utz6csm0jMQKTo-rG2Z8Wds0lY1frVr1yWmmoinsWLDx04BV-BXEQbxWCyb0JcJW1pQ31qJmQcdnSV6oD9mUY3aSEh_5eXKT-x4RRAwsmoSH-RsCfq-laTpK2lFlXqFmOMALgRLhAimYq8OKPw6vBxoxv-YQrfpbhJmIbKlB2_tdzmn16zZp4T20Aowjw7tpKaYJBTP69HYulIg3lp4', 'size': 16620}}


KeyError: 'data'

#### escrapear todo el texto del email

In [161]:
for email_ids in emailIdList_filter_by_title: 
    # Get the message from its id 
    txt = service.users().messages().get(userId='me', id=email_ids['id']).execute() 
  
    # Use try-except to avoid any Errors 
    # Get value of 'payload' from dictionary 'txt' 
    payload = txt['payload'] 
    headers = payload['headers'] 
  
    # Look for Subject and Sender Email in the headers 
    for d in headers: 
        if d['name'] == 'Subject': 
            subject = d['value'] 
        if d['name'] == 'From': 
            sender = d['value'] 
  
    # The Body of the message is in Encrypted format. So, we have to decode it. 
    # Get the data and decode it with base 64 decoder. 
    parts = payload.get('parts')[0] 
    data = parts['body']['data'] 
    data = data.replace("-","+").replace("_","/") 
    decoded_data = base64.b64decode(data) 
            
  
    # Now, the data obtained is in lxml. So, we will parse  
    # it with BeautifulSoup library 
    soup = BeautifulSoup(decoded_data , "lxml") 
    body = soup.body() 
  
    # Printing the subject, sender's email and message 
    print("Subject: ", subject) 
    print("From: ", sender) 
    print("Message: ", body) 
    print('\n') 

KeyError: 'data'

In [162]:
def getEmails(): 
    # Variable creds will store the user access token. 
    # If no valid token found, we will create one. 
    creds = None
  
    # The file token.pickle contains the user access token. 
    # Check if it exists 
    if os.path.exists('token.pickle'): 
  
        # Read the token from the file and store it in the variable creds 
        with open('token.pickle', 'rb') as token: 
            creds = pickle.load(token) 
  
    # If credentials are not available or are invalid, ask the user to log in. 
    if not creds or not creds.valid: 
        if creds and creds.expired and creds.refresh_token: 
            creds.refresh(Request()) 
        else: 
            flow = InstalledAppFlow.from_client_secrets_file(client_secret_file, SCOPES) 
            creds = flow.run_local_server(port=0) 
  
        # Save the access token in token.pickle file for the next run 
        with open('token.pickle', 'wb') as token: 
            pickle.dump(creds, token) 
  
    # Connect to the Gmail API 
    service = build('gmail', 'v1', credentials=creds) 
  
    # request a list of all the messages 
    result = service.users().messages().list(userId='me').execute() 
  
    # We can also pass maxResults to get any number of emails. Like this: 
    # result = service.users().messages().list(maxResults=200, userId='me').execute() 
    messages = result.get('messages') 
  
    # messages is a list of dictionaries where each dictionary contains a message id. 
  
    # iterate through all the messages 
    for msg in messages: 
        # Get the message from its id 
        txt = service.users().messages().get(userId='me', id=msg['id']).execute() 
  
        # Use try-except to avoid any Errors 
        try: 
            # Get value of 'payload' from dictionary 'txt' 
            payload = txt['payload'] 
            headers = payload['headers'] 
  
            # Look for Subject and Sender Email in the headers 
            for d in headers: 
                if d['name'] == 'Subject': 
                    subject = d['value'] 
                if d['name'] == 'From': 
                    sender = d['value'] 
  
            # The Body of the message is in Encrypted format. So, we have to decode it. 
            # Get the data and decode it with base 64 decoder. 
            parts = payload.get('parts')[0] 
            data = parts['body']['data'] 
            data = data.replace("-","+").replace("_","/") 
            decoded_data = base64.b64decode(data) 
            
  
            # Now, the data obtained is in lxml. So, we will parse  
            # it with BeautifulSoup library 
            soup = BeautifulSoup(decoded_data , "csv") 
            body = soup.body() 
  
            # Printing the subject, sender's email and message 
            print("Subject: ", subject) 
            print("From: ", sender) 
            print("Message: ", body) 
            print('\n') 
        except: 
            pass
  
  
getEmails()